In [ ]:
pip install openai

In [ ]:
import os
import time
import json
from openai import AzureOpenAI

# Set your API key and endpoint directly
api_key = "46bdeda9523d43b89c0c47de677fafab"
endpoint = "https://team-invincible.openai.azure.com/"


client = AzureOpenAI(
    api_key=api_key,
    api_version="2024-02-15-preview",
    azure_endpoint=endpoint
)

# Create the assistant with updated instructions
assistant = client.beta.assistants.create(
    instructions="""
    If a user interacts with you, the first thing you need to ask is a question regarding their skills, not how can I assist you today.
You are a data collector that asks for information about user skills, experience, project experience, and domain knowledge. At last, you will give a final output that includes the data you have collected and verify it with the user. and print the collected data as table

ask question one by one if user mentioned skills ask one by one skill information make short and sweet and ask user want to exit the chat every time if yes means ask them to enter exit

    """,
    model="gpt-35-turbo-16k",
    tools=[]
)




In [ ]:
# Loop for interaction
collected_messages = []
while True:
    # Create a new thread for each interaction
    thread = client.beta.threads.create()

    # Get user input
    user_input = input("You: ")

    # Add user input to the thread
    message = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=user_input
    )

    # Run the thread
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id
    )

    # Loop until the run completes or fails
    while run.status in ['queued', 'in_progress', 'cancelling']:
        time.sleep(1)
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id
        )

    # Check the status of the run and print messages if completed
    if run.status == 'completed':
        response = client.beta.threads.messages.list(
            thread_id=thread.id
        )
        for msg in response:
            if msg.role == 'assistant':
                print("Assistant:", msg.content)
                # Extract text content from TextContentBlock objects
                if isinstance(msg.content, list):
                    for content_block in msg.content:
                        if content_block.type == 'text':
                            collected_messages.append({"role": msg.role, "content": str(content_block.text)})
                else:
                    collected_messages.append({"role": msg.role, "content": str(msg.content.text)})
    elif run.status == 'requires_action':
        # The assistant requires calling some functions
        # and submit the tool outputs back to the run
        pass
    else:
        print(run.status)''

    # Check if user wants to exit the conversation
    if user_input.lower() == 'exit':
        break

# Print collected messages in JSON format
print(json.dumps(collected_messages, indent=4))

Assistant: [TextContentBlock(text=Text(annotations=[], value='Hello! Before we begin, I would like to ask you a few questions about your skills. Are you ready to proceed?'), type='text')]
Assistant: [TextContentBlock(text=Text(annotations=[], value="Great! Let's start with Java. How would you rate your skill level in Java on a scale of 1 to 10, with 1 being a beginner and 10 being an expert?"), type='text')]
Assistant: [TextContentBlock(text=Text(annotations=[], value='What are your skills?'), type='text')]
Assistant: [TextContentBlock(text=Text(annotations=[], value='Great! Can you please tell me about your experience with Spring Boot and Spring Batch?'), type='text')]
Assistant: [TextContentBlock(text=Text(annotations=[], value='Hello! Thank you for interacting with me. Before we proceed, may I ask what skills you have?'), type='text')]
Assistant: [TextContentBlock(text=Text(annotations=[], value='Great! It looks like you have mentioned Python as one of your skills. Can you please pr

KeyboardInterrupt: Interrupted by user